# Minimize and filter high energy conformers using openff force field
https://github.com/openforcefield/openff-toolkit/tree/de8a4a545351301adfe424dff0d879b2dd13bc0b/examples

In [1]:
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.utils import RDKitToolkitWrapper, get_data_file_path
from rdkit import Chem
from rdkit.Chem import rdMolAlign
import numpy as np
from simtk import openmm, unit
from openff.units import unit as off_unit
import shutil

In [3]:
def minimize(filename, molname):
    # Load sdf
    loaded_molecules = Molecule.from_file(filename)
        
        
    # Collatate all conformers of the same molecule
    # NOTE: This isn't necessary if you have already loaded or created multi-conformer molecules;
    # it is just needed because our SDF reader does not automatically collapse conformers.
    molecules = [loaded_molecules[0]]
    for i, molecule in enumerate(loaded_molecules[1:]):
        # add conformer to existing molecule
        if molecule == molecules[-1]:
            for conformer in molecule.conformers:
                molecules[-1].add_conformer(conformer)
        else:
            # add new molecule
            molecules.append(molecule)
    n_molecules = len(molecules)
    n_conformers = sum([mol.n_conformers for mol in molecules])
    print(f'{n_molecules} unique molecule(s) loaded, with {n_conformers} total conformers')
    
    
    # Load the openff-2.0.0 force field appropriate for vacuum calculations (with constraints)
    from openff.toolkit.typing.engines.smirnoff import ForceField
    #forcefield = ForceField('openff_unconstrained-2.0.0.offxml')
    forcefield = ForceField('openff-2.0.0.offxml')
    

    # Loop over molecules and minimize each conformer
    for molecule in molecules:
        molecule.name = molname
        print('%s : %d conformers' % (molecule.name, molecule.n_conformers))

        # Make a temporary copy of the molecule that we can update for each minimization
        mol_copy = Molecule(molecule)

        # Make a copy of the molecule to store minimized structures
        mol_copy_min = Molecule(molecule)
        mol_copy_min._conformers = None

        # Make an OpenFF Topology so we can parameterize the system
        off_top = molecule.to_topology()
        print(f"Parametrizing {molecule.name} (may take a moment to calculate charges)")
        system = forcefield.create_openmm_system(off_top)

        # Use OpenMM to compute initial and minimized energy for all conformers
        integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
        platform = openmm.Platform.getPlatformByName('Reference')
        omm_top = off_top.to_openmm()
        simulation = openmm.app.Simulation(omm_top, system, integrator, platform)

        # Print text header
        print('Conformer\tInitial PE\tMinimized PE\tRMS between initial and minimized conformer')
        output = [['Conformer','Initial PE (kcal/mol)','Minimized PE (kcal/mol)','RMS between initial and minimized conformer (Angstrom)']]
        
        # Loop over conformers
        energies = []
        for conformer_index, conformer in enumerate(molecule.conformers):
            omm_conformer = conformer.to_openmm()
            simulation.context.setPositions(omm_conformer.in_units_of(unit.nanometer))
            orig_potential = simulation.context.getState(getEnergy=True).getPotentialEnergy()
            simulation.minimizeEnergy(maxIterations=10)
            min_state = simulation.context.getState(getEnergy=True, getPositions=True)
            min_potential = min_state.getPotentialEnergy()
            energies.append(min_potential/unit.kilocalories_per_mole)
            
            # Calculate the RMSD between the initial and minimized conformer
            min_coords = min_state.getPositions()
            min_coords = np.array([ [atom.x, atom.y, atom.z] for atom in min_coords]) * unit.nanometer
            mol_copy._conformers = None
            mol_copy.add_conformer(conformer)
            mol_copy.add_conformer(min_coords)
            rdmol = mol_copy.to_rdkit()
            rmslist = []
            rdMolAlign.AlignMolConformers(rdmol, RMSlist=rmslist)
            minimization_rms = rmslist[0]

            # Save report
            mol_copy_min.add_conformer(min_coords)
            print('%5d\t%5d\t%8.3f\t%8.3f\t%8.3f' % (conformer_index+1, molecule.n_conformers, orig_potential/unit.kilocalories_per_mole, min_potential/unit.kilocalories_per_mole, minimization_rms))
            output.append([str(conformer_index+1), 
                           f'{orig_potential/unit.kilocalories_per_mole:.3f}', 
                           f'{min_potential/unit.kilocalories_per_mole:.3f}', 
                           f'{minimization_rms:.3f}'])

        # Write the results out to CSV
        with open(f'{molecule.name}.csv', 'w') as of:
            for line in output:
                of.write(','.join(line)+'\n')

        # Save (this doesn't save all conformations?)
        #mol_copy_min.to_file(f'{molecule.name}_conf_minimized.sdf', file_format='sdf')

        # Clean up OpenMM Simulation
        del simulation, integrator
        
        # Relative energy
        e = np.array(energies)
        e = e - e.min()
        
    return mol_copy_min, e

#### run

In [4]:
threshold = 62.75  # 0.1 hartee

In [5]:
filename = "torsion_scan_a.sdf"
molname = "adenosine"

offmols, energies = minimize(filename, molname)
rdmols = offmols.to_rdkit()

count = 0
filename_basename = filename.split('.')[0]
w = Chem.SDWriter(f'{filename_basename}_filtered.sdf')
for cid in range(rdmols.GetNumConformers()):
    if energies[cid] < threshold:
        rdmols.SetProp('ID', f'conf_{cid+1}')
        rdmols.SetProp('RelativeEnergy', f'{energies[cid]}')
        w.write(rdmols, confId=cid)
    else:
        count += 1

1 unique molecule(s) loaded, with 2400 total conformers
adenosine : 2400 conformers
Parametrizing adenosine (may take a moment to calculate charges)
Conformer	Initial PE	Minimized PE	RMS between initial and minimized conformer
    1	 2400	 -71.296	 -79.133	   0.049
    2	 2400	 -73.086	 -80.447	   0.048
    3	 2400	 -75.448	 -82.380	   0.047
    4	 2400	 -77.849	 -84.584	   0.047
    5	 2400	 -79.864	 -86.583	   0.048
    6	 2400	 -81.057	 -87.856	   0.047
    7	 2400	 -79.617	 -87.733	   0.052
    8	 2400	 -68.351	 -86.020	   0.065
    9	 2400	 -50.082	 -84.461	   0.076
   10	 2400	 -60.955	 -86.065	   0.069
   11	 2400	 -79.380	 -89.438	   0.055
   12	 2400	 -84.837	 -91.653	   0.047
   13	 2400	 -85.314	 -92.203	   0.050
   14	 2400	 -84.884	 -91.936	   0.051
   15	 2400	 -84.383	 -91.500	   0.050
   16	 2400	 -83.734	 -90.929	   0.050
   17	 2400	 -82.621	 -89.916	   0.050
   18	 2400	 -80.596	 -88.194	   0.053
   19	 2400	 -76.777	 -85.833	   0.065
   20	 2400	 -67.966	 -82.919	  

In [6]:
print(f'{count}/{rdmols.GetNumConformers()} failed')

7/2400 failed


In [7]:
filename = "torsion_scan_c.sdf"
molname = "cytidine"

offmols, energies = minimize(filename, molname)
rdmols = offmols.to_rdkit()

count = 0
filename_basename = filename.split('.')[0]
w = Chem.SDWriter(f'{filename_basename}_filtered.sdf')
for cid in range(rdmols.GetNumConformers()):
    if energies[cid] < threshold:
        rdmols.SetProp('ID', f'conf_{cid+1}')
        rdmols.SetProp('RelativeEnergy', f'{energies[cid]}')
        w.write(rdmols, confId=cid)
    else:
        count += 1

1 unique molecule(s) loaded, with 2400 total conformers
cytidine : 2400 conformers
Parametrizing cytidine (may take a moment to calculate charges)
Conformer	Initial PE	Minimized PE	RMS between initial and minimized conformer
    1	 2400	 -47.560	-101.791	   0.095
    2	 2400	 -54.487	-103.114	   0.093
    3	 2400	 -72.069	-105.407	   0.087
    4	 2400	 -89.930	-109.145	   0.080
    5	 2400	 -90.887	-110.714	   0.073
    6	 2400	 -68.017	-111.189	   0.084
    7	 2400	 -76.411	-112.187	   0.083
    8	 2400	 -76.753	-111.760	   0.083
    9	 2400	  32.022	-110.621	   0.109
   10	 2400	 194.794	-109.300	   0.120
   11	 2400	  22.603	-113.783	   0.103
   12	 2400	 -93.898	-118.853	   0.078
   13	 2400	-112.805	-121.276	   0.051
   14	 2400	-114.967	-122.318	   0.049
   15	 2400	-114.684	-121.815	   0.047
   16	 2400	-113.408	-120.579	   0.046
   17	 2400	-111.485	-118.861	   0.046
   18	 2400	-109.571	-116.839	   0.045
   19	 2400	-107.941	-114.840	   0.044
   20	 2400	-105.482	-112.187	   0

In [8]:
print(f'{count}/{rdmols.GetNumConformers()} failed')

18/2400 failed


In [9]:
filename = "torsion_scan_g.sdf"
molname = "guanosine"

offmols, energies = minimize(filename, molname)
rdmols = offmols.to_rdkit()

count = 0
filename_basename = filename.split('.')[0]
w = Chem.SDWriter(f'{filename_basename}_filtered.sdf')
for cid in range(rdmols.GetNumConformers()):
    if energies[cid] < threshold:
        rdmols.SetProp('ID', f'conf_{cid+1}')
        rdmols.SetProp('RelativeEnergy', f'{energies[cid]}')
        w.write(rdmols, confId=cid)
    else:
        count += 1

1 unique molecule(s) loaded, with 2400 total conformers
guanosine : 2400 conformers
Parametrizing guanosine (may take a moment to calculate charges)
Conformer	Initial PE	Minimized PE	RMS between initial and minimized conformer
    1	 2400	 -26.570	 -39.810	   0.068
    2	 2400	 -28.886	 -41.613	   0.070
    3	 2400	 -31.973	 -43.488	   0.068
    4	 2400	 -33.817	 -45.151	   0.065
    5	 2400	 -35.644	 -46.866	   0.063
    6	 2400	 -37.122	 -47.470	   0.058
    7	 2400	 -29.859	 -46.137	   0.067
    8	 2400	 -20.700	 -45.421	   0.074
    9	 2400	 -29.122	 -47.936	   0.072
   10	 2400	 -39.164	 -51.197	   0.060
   11	 2400	 -43.508	 -53.424	   0.052
   12	 2400	 -45.318	 -54.318	   0.049
   13	 2400	 -44.960	 -53.831	   0.050
   14	 2400	 -43.364	 -52.164	   0.051
   15	 2400	 -41.895	 -50.567	   0.051
   16	 2400	 -40.808	 -49.443	   0.051
   17	 2400	 -39.790	 -48.420	   0.052
   18	 2400	 -38.435	 -47.032	   0.050
   19	 2400	 -36.478	 -45.116	   0.051
   20	 2400	 -34.447	 -43.212	  

In [10]:
print(f'{count}/{rdmols.GetNumConformers()} failed')

8/2400 failed


In [11]:
filename = "torsion_scan_u.sdf"
molname = "uridine"

offmols, energies = minimize(filename, molname)
rdmols = offmols.to_rdkit()

count = 0
filename_basename = filename.split('.')[0]
w = Chem.SDWriter(f'{filename_basename}_filtered.sdf')
for cid in range(rdmols.GetNumConformers()):
    if energies[cid] < threshold:
        rdmols.SetProp('ID', f'conf_{cid+1}')
        rdmols.SetProp('RelativeEnergy', f'{energies[cid]}')
        w.write(rdmols, confId=cid)
    else:
        count += 1

1 unique molecule(s) loaded, with 2400 total conformers
uridine : 2400 conformers
Parametrizing uridine (may take a moment to calculate charges)
Conformer	Initial PE	Minimized PE	RMS between initial and minimized conformer
    1	 2400	 -47.047	 -73.608	   0.079
    2	 2400	 -54.188	 -75.744	   0.076
    3	 2400	 -62.768	 -77.944	   0.068
    4	 2400	 -70.683	 -81.357	   0.061
    5	 2400	 -77.477	 -83.917	   0.043
    6	 2400	 -76.216	 -84.105	   0.052
    7	 2400	 -45.382	 -82.414	   0.092
    8	 2400	 149.031	 -78.253	   0.124
    9	 2400	 382.667	 -73.629	   0.141
   10	 2400	  94.576	 -74.071	   0.106
   11	 2400	 -49.423	 -77.259	   0.079
   12	 2400	 -69.210	 -79.207	   0.061
   13	 2400	 -71.767	 -80.362	   0.056
   14	 2400	 -73.549	 -81.212	   0.053
   15	 2400	 -75.305	 -81.803	   0.046
   16	 2400	 -76.193	 -82.190	   0.041
   17	 2400	 -76.201	 -82.125	   0.040
   18	 2400	 -74.982	 -80.977	   0.042
   19	 2400	 -70.529	 -78.894	   0.063
   20	 2400	 -52.118	 -75.955	   0.0

In [12]:
print(f'{count}/{rdmols.GetNumConformers()} failed')

12/2400 failed
